In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import ssl
from scraper import scrape

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/pvpswaghd/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores

# Example usage

scraped_lst = scrape("tesla", 10)
print(scraped_lst)


[['SNAPSHOT Wall St slides after hotter-than-expected producer prices data', "Aug 11 (Reuters) - Wall Street's main indexes opened lower on Friday as stronger-than-expected producer prices data lifted U.S. bond yields, weighing down rate-sensitive megacap technology and growth stocks.The Dow Jones Industrial Average (.DJI) fell 64.79 points, or 0.18%, at the open to 35,111.36.The S&P 500 (.SPX) opened lower by 18.14 points, or 0.41%, at 4,450.69, while the Nasdaq Composite (.IXIC) dropped 104.34 points, or 0.76%, to 13,633.65 at the opening bell."], ["BYD calls on China automakers to unite, 'demolish the old' in global push", 'Aug 11 (Reuters) - A patriotic call by China\'s bestselling automaker to band the industry together and "demolish the old legends" of the global market has gone viral, drawing both raves and a rebuke from a rival.BYD (002594.SZ) used an event this week to mark a production milestone to celebrate a bigger purpose: the emergence of China as a global auto manufactur

In [4]:
# Load model directly
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import spacy

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [14]:
import numpy as np
nlp = spacy.load("en_core_web_lg")
count = 0
score = []
print(scraped_lst)
for i in scraped_lst:
    if analyze_sentiment(i[0]+i[1])["compound"] > 0.97:
        print("Topic:", i[0])
    score.append(analyze_sentiment(i[0]+i[1])['compound'])
volatility = np.std(score)
print(volatility)
print(score)
    

[['SNAPSHOT Wall St slides after hotter-than-expected producer prices data', "Aug 11 (Reuters) - Wall Street's main indexes opened lower on Friday as stronger-than-expected producer prices data lifted U.S. bond yields, weighing down rate-sensitive megacap technology and growth stocks.The Dow Jones Industrial Average (.DJI) fell 64.79 points, or 0.18%, at the open to 35,111.36.The S&P 500 (.SPX) opened lower by 18.14 points, or 0.41%, at 4,450.69, while the Nasdaq Composite (.IXIC) dropped 104.34 points, or 0.76%, to 13,633.65 at the opening bell."], ["BYD calls on China automakers to unite, 'demolish the old' in global push", 'Aug 11 (Reuters) - A patriotic call by China\'s bestselling automaker to band the industry together and "demolish the old legends" of the global market has gone viral, drawing both raves and a rebuke from a rival.BYD (002594.SZ) used an event this week to mark a production milestone to celebrate a bigger purpose: the emergence of China as a global auto manufactur

In [18]:
import openai

openai.api_key = "j5-eGSjrJEqe5mGZz_q4JA%3D%3"

model_id = 'gpt-3.5-turbo'
def ChatGPT_conversation(conversation):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=conversation
    )
    # api_usage = response['usage']
    # print('Total token consumed: {0}'.format(api_usage['total_tokens']))
    # stop means complete
    # print(response['choices'][0].finish_reason)
    # print(response['choices'][0].index)
    conversation.append({'role': response.choices[0].message.role, 'content': response.choices[0].message.content})
    return conversation

conversation = []
conversation.append({'role': 'system', 'content': 'How may I help you?'})
conversation = ChatGPT_conversation(conversation)
print('{0}: {1}\n'.format(conversation[-1]['role'].strip(), conversation[-1]['content'].strip()))

while True:
    prompt = input('User:')
    conversation.append({'role': 'user', 'content': prompt})
    conversation = ChatGPT_conversation(conversation)
    print('{0}: {1}\n'.format(conversation[-1]['role'].strip(), conversation[-1]['content'].strip()))

AuthenticationError: Incorrect API key provided: j5-eGSjr***************3D%3. You can find your API key at https://platform.openai.com/account/api-keys.